In [1]:
library(zellkonverter)
library(SingleCellExperiment)
library(Seurat)
library(Azimuth)
library(tidyverse)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
library(irlba)
library(harmony)
library(symphony)

Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


The following object is masked from 'package:S4Vectors':

    expand


Loading required package: Rcpp



# convert AIM-seq h5ad to seurat

In [4]:
counts_fn = '../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.h5ad'

In [6]:
# Load the .h5ad file into a SingleCellExperiment object
sce = readH5AD(counts_fn)

+ /PHShome/mc1070/.cache/R/basilisk/1.10.2/0/bin/conda create --yes --prefix /PHShome/mc1070/.cache/R/basilisk/1.10.2/zellkonverter/1.8.0/zellkonverterAnnDataEnv-0.8.0 'python=3.8.13' --quiet -c conda-forge

+ /PHShome/mc1070/.cache/R/basilisk/1.10.2/0/bin/conda install --yes --prefix /PHShome/mc1070/.cache/R/basilisk/1.10.2/zellkonverter/1.8.0/zellkonverterAnnDataEnv-0.8.0 'python=3.8.13' -c conda-forge

+ /PHShome/mc1070/.cache/R/basilisk/1.10.2/0/bin/conda install --yes --prefix /PHShome/mc1070/.cache/R/basilisk/1.10.2/zellkonverter/1.8.0/zellkonverterAnnDataEnv-0.8.0 -c conda-forge 'python=3.8.13' 'anndata=0.8.0' 'h5py=3.6.0' 'hdf5=1.12.1' 'natsort=8.1.0' 'numpy=1.22.3' 'packaging=21.3' 'pandas=1.4.2' 'python=3.8.13' 'scipy=1.7.3' 'sqlite=3.38.2'



In [ ]:
# Save as a seurat object (RDS file)

In [13]:
assayNames(sce)

[1] "X"

In [62]:
sce %>% str(3)

Formal class 'SingleCellExperiment' [package "SingleCellExperiment"] with 9 slots
  ..@ int_elementMetadata:Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  ..@ int_colData        :Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  ..@ int_metadata       :List of 1
  .. ..$ version:Classes 'package_version', 'numeric_version'  hidden list of 1
  ..@ rowRanges          :Formal class 'CompressedGRangesList' [package "GenomicRanges"] with 5 slots
  ..@ colData            :Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  ..@ assays             :Formal class 'SimpleAssays' [package "SummarizedExperiment"] with 1 slot
  ..@ NAMES              : NULL
  ..@ elementMetadata    :Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  ..@ metadata           : list()


In [54]:
saveRDS(sce, '../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.SCE.rds')

In [32]:
sce

class: SingleCellExperiment 
dim: 36738 43222 
metadata(0):
assays(1): X
rownames(36738): ENSG00000000003 ENSG00000000005 ...
  TIGIT_or_VSTM3_prot integrinb7_prot
rowData names(3): gene_sybmol gene_name feature_type
colnames(43222): 3_AAACCTGAGACCGGAT 3_AAACCTGAGAGGGCTT ...
  2_TTTGTCATCGGCTACG 2_TTTGTCATCTCCAGGG
colData names(72): barcode BRI ... Hashtag2 Hashtag3
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [78]:
sce_renamed = sce

In [80]:
rownames(sce_renamed) <- rowData(sce_renamed)$gene_name %>% unfactor %>% make.unique

In [81]:
rownames(rowData(sce_renamed)) = rownames(sce_renamed) #rowData(sce_renamed)[['gene_name']]

In [47]:
# rownames(sce_renamed) = rowData(sce_renamed)[['gene_name']]

In [58]:
rowData(sce_renamed)[['gene_name']] %>% length

[1] 36738

In [82]:
sce_renamed

class: SingleCellExperiment 
dim: 36738 43222 
metadata(0):
assays(1): X
rownames(36738): TSPAN6 TNMD ... TIGIT_or_VSTM3_prot integrinb7_prot
rowData names(3): gene_sybmol gene_name feature_type
colnames(43222): 3_AAACCTGAGACCGGAT 3_AAACCTGAGAGGGCTT ...
  2_TTTGTCATCGGCTACG 2_TTTGTCATCTCCAGGG
colData names(72): barcode BRI ... Hashtag2 Hashtag3
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [83]:
seu = as.Seurat(sce_renamed, counts = "X", data = 'X')

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [84]:
seu

An object of class Seurat 
36738 features across 43222 samples within 1 assay 
Active assay: originalexp (36738 features, 0 variable features)
 2 layers present: counts, data

In [85]:
seu %>% str

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 1
  .. ..$ originalexp:Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:97201453] 3 18 36 38 45 55 90 106 110 124 ...
  .. .. .. .. .. ..@ p       : int [1:43223] 0 1085 2472 6999 8736 11941 16304 18080 19183 22006 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 36738 43222
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:36738] "TSPAN6" "TNMD" "DPM1" "SCYL3" ...
  .. .. .. .. .. .. ..$ : chr [1:43222] "3_AAACCTGAGACCGGAT" "3_AAACCTGAGAGGGCTT" "3_AAACCTGAGAGTCTGG" "3_AAACCTGAGCTAGCCC" ...
  .. .. .. .. .. ..@ x       : num [1:97201453] 2 1 1 1 1 1 1 1 1 1 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:97201453] 3 18 36 38 45 55 90

In [87]:
saveRDS(seu, '../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.Seurat.rds')

# run azimuth

In [89]:
az_res = RunAzimuth(query = seu, reference = "pbmcref")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message:
"Overwriting miscellanous data for model"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
Warning message:
"Adding a dimensional reduction (refUMAP) without the associated assay being present"
detected inputs from HUMAN with id type Gene.name

reference rownames detected HUMAN with id type Gene.name

Normalizing query using reference SCT model

Warning message:
"113 features of the features specified were not present in both the reference query assays. 
Continuing with remaining 4887 features."
Projecting cell embeddings

Finding query neighbors

Finding neighborhoods

Finding anchors

	Found 10539 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Predicting cell labels

War

In [91]:
az_res@meta.data %>% write.table('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.Azimuth.tsv',
                              row.names = TRUE, sep = '\t')

# run symphony

## convert Hao h5ad to seurat

In [93]:
counts_fn = '../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.h5ad'

In [94]:
# Load the .h5ad file into a SingleCellExperiment object
sce = readH5AD(counts_fn)

In [95]:
sce

class: SingleCellExperiment 
dim: 20957 73259 
metadata(0):
assays(1): X
rownames(20957): AL627309.1 AL669831.5 ... AB_CD202b AB_CD11c
rowData names(7): features Clone ... Gene_ADT_Fixed Clone_ADT_Fixed
colnames(73259): L1_AAACCCAAGACATACA L1_AAACCCACAACTGGTT ...
  E2L8_TTTGTTGCAGCGTGAA E2L8_TTTGTTGGTCGTGATT
colData names(15): nCount_ADT nFeature_ADT ... nFeature_SCT
  celltype.l3.merged
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [146]:
rna_features = rownames(sce)[-grep("^AB_", rownames(sce))]  # Adjust pattern based on your data
adt_features = rownames(sce)[grep("^AB_", rownames(sce))]  # Adjust pattern based on your data

In [148]:
rna_assay = assay(sce)[rna_features, ]
adt_assay = assay(sce)[adt_features, ]

In [154]:
seu = CreateSeuratObject(counts = rna_assay, assay = "RNA", meta.data = colData(sce))

Warning message:
"[[<- defined for objects of type "S4" only for subclasses of environment"


In [155]:
seu[["ADT"]] = CreateAssayObject(counts = adt_assay)


Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [178]:
seu@meta.data = colData(sce) %>% as.data.frame

In [181]:
seu

An object of class Seurat 
20957 features across 73259 samples within 2 assays 
Active assay: RNA (20729 features, 0 variable features)
 1 layer present: counts
 1 other assay present: ADT

In [182]:
saveRDS(seu, '../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.Seurat.rds')

## create symphony ref

In [209]:
seu = readRDS('../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.Seurat.rds')

In [210]:
seu

An object of class Seurat 
20957 features across 73259 samples within 2 assays 
Active assay: RNA (20729 features, 0 variable features)
 1 layer present: counts
 1 other assay present: ADT

In [212]:
exprs = GetAssayData(object = seu, assay = "RNA", slot = "counts")

Warning message:
"The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead."


In [214]:
exprs %>% dim

[1] 20729 73259

In [216]:
ref_exp_full = exprs %>% singlecellmethods::normalizeData(method = "log")

In [220]:
var_genes = vargenes_vst(ref_exp_full, groups = as.character(seu@meta.data[['donor']]), topn = 2000)
ref_exp = ref_exp_full[var_genes, ]
dim(ref_exp)

[1]  6101 73259

In [221]:
vargenes_means_sds = tibble(symbol = var_genes, mean = Matrix::rowMeans(ref_exp))
vargenes_means_sds$stddev = singlecellmethods::rowSDs(ref_exp, vargenes_means_sds$mean)
head(vargenes_means_sds)

symbol,mean,stddev
<chr>,<dbl>,<dbl>
GNLY,0.7554823,1.3009975
CCL5,1.1372781,1.3605393
S100A4,1.8824635,0.9632974
NKG7,0.9064792,1.2809740
LTB,1.8663526,0.8887139
IL32,1.8167456,0.7743503


In [222]:
ref_exp_scaled = singlecellmethods::scaleDataWithStats(ref_exp, vargenes_means_sds$mean, vargenes_means_sds$stddev, 1)

In [225]:
set.seed(0)
s = irlba(ref_exp_scaled, nv = 20)
Z_pca_ref = diag(s$d) %*% t(s$v) # [pcs by cells]
loadings = s$u

In [228]:
harmony_vars

[1] "lane"  "donor"

In [230]:
set.seed(0)
ref_harmObj = harmony::HarmonyMatrix(
        data_mat = t(Z_pca_ref),  ## PCA embedding matrix of cells
        meta_data = seu@meta.data, ## dataframe with cell labels
        # theta = c(2),             ## cluster diversity enforcement
        vars_use = harmony_vars,    ## variable to integrate out
        nclust = 100,             ## number of clusters in Harmony model
        max.iter.harmony = 20,
        return_object = TRUE,     ## return the full Harmony model object
        do_pca = FALSE            ## don't recompute PCs
)

Warning message:
"HarmonyMatrix is deprecated and will be removed in the future from the API in the future"
Transposing data matrix

Initializing state using k-means centroids initialization

Harmony 1/10

Harmony 2/10

Harmony 3/10

Harmony 4/10

Harmony converged after 4 iterations



In [232]:
getwd()

[1] "/data/srlab1/TCAT/Analysis/PerDataset/Aimseq"

In [233]:
# Compress a Harmony object into a Symphony reference
reference = symphony::buildReferenceFromHarmonyObj(
                           ref_harmObj,            # output object from HarmonyMatrix()
                           seu@meta.data,           # reference cell metadata
                           vargenes_means_sds,     # gene names, means, and std devs for scaling
                           loadings,               # genes x PCs matrix
                           verbose = TRUE,         # verbose output
                           do_umap = TRUE,         # Set to TRUE only when UMAP model was saved for reference
                           save_uwot_path = '../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.symphony.uwot'
)


Save metadata, vargenes (S), and loadings (U)

Save R, Z_orig, Z_corr, and betas from Harmony object

Calculate final L2 normalized reference centroids (Y_cos)

Calculate reference compression terms (Nr and C)

UMAP

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

Warning message:
"invalid uid value replaced by that for user 'nobody'"
Saved uwot model

Finished nicely.



In [238]:
saveRDS(reference, '../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.symphony.rds')

## map query

In [2]:
reference = readRDS('../../../Data/PerDataset/HaoEtAl/haoetal_pbmc_multimodal.merged.T.raw.ADTfixed.symphony.rds')

In [3]:
reference %>% str

List of 12
 $ meta_data     :'data.frame':	73259 obs. of  15 variables:
  ..$ nCount_ADT        : num [1:73259] 5949 6547 6318 5195 4501 ...
  ..$ nFeature_ADT      : int [1:73259] 211 217 219 213 210 216 212 219 209 214 ...
  ..$ nCount_RNA        : num [1:73259] 5864 5067 6505 4332 3401 ...
  ..$ nFeature_RNA      : int [1:73259] 1617 1381 1621 1633 1200 1568 1318 1451 1390 1255 ...
  ..$ orig.ident        : Factor w/ 24 levels "P1_0","P1_3",..: 3 11 12 8 8 7 1 5 1 9 ...
  ..$ lane              : Factor w/ 13 levels "E2L1","E2L2",..: 9 9 9 9 9 9 9 9 9 9 ...
  ..$ donor             : Factor w/ 8 levels "P1","P2","P3",..: 1 4 4 3 3 3 1 2 1 3 ...
  ..$ time              : Factor w/ 3 levels "0","3","7": 3 2 3 2 2 1 1 2 1 3 ...
  ..$ celltype.l1       : Factor w/ 3 levels "CD4 T","CD8 T",..: 1 2 2 2 2 1 1 1 1 3 ...
  ..$ celltype.l2       : Factor w/ 13 levels "CD4 CTL","CD4 Naive",..: 4 6 6 9 6 2 4 1 4 10 ...
  ..$ celltype.l3       : Factor w/ 31 levels "CD4 CTL","CD4 Naive",..: 4 11 1

In [4]:
query_seu = readRDS('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.Seurat.rds')

In [5]:
query_exprs = GetAssayData(object = query_seu, assay = "originalexp", slot = "counts")


Warning message:
"The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead."


In [6]:
rna_features = rownames(query_exprs)[-grep("-prot", rownames(query_exprs))]  

query_exprs = query_exprs[rna_features, ]

query_exprs_norm = query_exprs %>% singlecellmethods::normalizeData(method = "log")

In [8]:
query = mapQuery(query_exprs_norm,             # query gene expression (genes x cells)
                 query_seu@meta.data,        # query metadata (cells x attributes)
                 reference,             # Symphony reference object
                 do_normalize = FALSE,  # perform log(CP10k+1) normalization on query
                 do_umap = TRUE)        # project query cells into reference UMAP

Scaling and synchronizing query gene expression

Found 5897 out of 6101 reference variable genes in query dataset

Project query cells using reference gene loadings

Clustering query cells to reference centroids

Correcting query batch effects

UMAP

All done!



In [10]:
query = knnPredict(query, reference, reference$meta_data$`celltype.l2`, k = 5)

In [15]:
query$meta_data[c('cell_type_pred_knn', 'cell_type_pred_knn_prob')]  %>% head

,cell_type_pred_knn,cell_type_pred_knn_prob
,<fct>,<dbl>
3_AAACCTGAGACCGGAT,CD4 TEM,0.6
3_AAACCTGAGAGGGCTT,CD4 Naive,1.0
3_AAACCTGAGAGTCTGG,CD4 TCM,1.0
3_AAACCTGAGCTAGCCC,CD4 Naive,0.6
3_AAACCTGAGCTGCCCA,CD4 TCM,1.0
3_AAACCTGAGGCTCATT,CD4 TCM,1.0


In [16]:
query$meta_data[c('cell_type_pred_knn', 'cell_type_pred_knn_prob')] %>% write.table('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.symphony.tsv',
                              row.names = TRUE, sep = '\t')

In [17]:
# Sync the column names for both data frames
reference$meta_data$cell_type_pred_knn = NA
reference$meta_data$cell_type_pred_knn_prob = NA
reference$meta_data$ref_query = 'reference'
query$meta_data$ref_query = 'query'



In [31]:
# Add the UMAP coordinates to the metadata
meta_data_combined = rbind(query$meta_data[c('cell_type_pred_knn', 'ref_query')] %>% rename('celltype.l2' = 'cell_type_pred_knn'), 
                           reference$meta_data[c('celltype.l2', 'ref_query')])


umap_combined = rbind(query$umap, reference$umap$embedding)


In [32]:
umap_combined_labels = cbind(meta_data_combined, umap_combined)

# run ProjecTILs

In [1]:
library(Seurat)
library(dplyr)
# library(tidyverse)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
library(ProjecTILs)


In [7]:
options(timeout = 600)

In [3]:
cd4ref = readRDS('../../../Data/Misc/CD4T_human_ref_v1.rds')
cd8ref = readRDS('../../../Data/Misc/CD8T_human_ref_v1.rds')

In [24]:
alltref = readRDS('../../../Data/Misc/ref_TILAtlas_mouse_v1.rds')

In [14]:
query_seu = readRDS('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.Seurat.rds')

In [15]:
query_exprs = GetAssayData(object = query_seu, assay = "originalexp", slot = "counts")
rna_features = rownames(query_exprs)[-grep("-prot", rownames(query_exprs))]  
query_exprs = query_exprs[rna_features, ]

In [26]:
query_rna = CreateSeuratObject(query_exprs, meta.data = query_seu@meta.data)

In [16]:
query_rna

An object of class Seurat 
36601 features across 43222 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)
 1 layer present: counts

In [17]:
query_cd4 = Run.ProjecTILs(query_rna, ref=cd4ref, filter.cells = FALSE)


  |                                                                      |   0%[1] "Using assay RNA for query"


Normalizing layer: counts



[1] "Aligning query to reference map for batch-correction..."


Warning message:
“Layer counts isn't present in the assay object[[assay]]; returning NULL”
Preparing PCA embeddings for objects...




Projecting corrected query onto Reference PCA space

Projecting corrected query onto Reference UMAP space
  |======================================================================| 100%



Creating slots functional.cluster and functional.cluster.conf in query object



In [21]:
query_cd4 %>% str

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 2
  .. ..$ RNA       :Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:91961659] 23 32 65 116 120 140 148 209 228 244 ...
  .. .. .. .. .. ..@ p       : int [1:43223] 0 966 2243 6646 8263 11343 15573 17241 18228 20926 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 50753 43222
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:50753] "A1BG" "A1BG-AS1" "A1CF" "A2M" ...
  .. .. .. .. .. .. ..$ : chr [1:43222] "3_AAACCTGAGACCGGAT" "3_AAACCTGAGAGGGCTT" "3_AAACCTGAGAGTCTGG" "3_AAACCTGAGCTAGCCC" ...
  .. .. .. .. .. ..@ x       : num [1:91961659] 1 1 2 1 1 1 1 1 1 1 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:91961659] 23 32 65 116 120

In [23]:
query_cd4@meta.data %>% head(3)

,orig.ident,nCount_RNA,nFeature_RNA,nCount_originalexp,nFeature_originalexp,barcode,BRI,Run,Well,Batch,⋯,A_Num_Productive_In_Barcode,A_Has_Unproductive,B_Possible_TCR_Doublet,A_Possible_TCR_Doublet,DEMUX_BEST_Category,Hashtag1,Hashtag2,Hashtag3,functional.cluster,functional.cluster.conf
,<fct>,<dbl>,<int>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<chr>,<dbl>
3_AAACCTGAGACCGGAT,3,1443,966,3241,1085,AAACCTGAGACCGGAT-1,2248_hashing,2,1,3,⋯,NaN,nan,nan,nan,SNG,8,19,4305,CD4.Tfh,0.7880882
3_AAACCTGAGAGGGCTT,3,3508,1277,4559,1387,AAACCTGAGAGGGCTT-1,2248_hashing,2,1,3,⋯,1,True,Productive Singleton,Productive Singleton,SNG,391,61,138,CD4.NaiveLike,1.0000000
3_AAACCTGAGAGTCTGG,3,18799,4403,20879,4527,AAACCTGAGAGTCTGG-1,2248_hashing,2,1,3,⋯,1,False,Productive Singleton,Productive Singleton,SNG,9,55,5301,CD4.Th17,0.9560450


In [24]:
query_cd8 = Run.ProjecTILs(query_rna, ref=cd8ref, filter.cells = FALSE)

  |                                                                      |   0%[1] "Using assay RNA for query"


Normalizing layer: counts



[1] "Aligning query to reference map for batch-correction..."


Warning message:
“Layer counts isn't present in the assay object[[assay]]; returning NULL”
Preparing PCA embeddings for objects...

Alignment failed due to: Error in getGlobalsAndPackages(expr, envir = envir, globals = globals): The total size of the 7 globals exported for future expression (‘FUN()’) is 590.15 MiB.. This exceeds the maximum allowed size of 500.00 MiB (option 'future.globals.maxSize'). The three largest globals are ‘FUN’ (563.83 MiB of class ‘function’), ‘query’ (17.64 MiB of class ‘numeric’) and ‘index’ (8.67 MiB of class ‘S4’)
 





[1] "DIRECTLY projecting query onto Reference PCA space"


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 11.8 GiB”


[1] "DIRECTLY projecting query onto Reference UMAP space"
  |======================================================================| 100%



Creating slots functional.cluster and functional.cluster.conf in query object



In [28]:
query_allt = Run.ProjecTILs(query_rna, ref=alltref, filter.cells = FALSE)

  |                                                                      |   0%[1] "Using assay RNA for query"


Normalizing layer: counts



[1] "Transforming expression matrix into space of orthologs"
[1] "Aligning query to reference map for batch-correction..."


Warning message:
“Layer counts isn't present in the assay object[[assay]]; returning NULL”
Warning message:
“Layer counts isn't present in the assay object[[assay]]; returning NULL”
Preparing PCA embeddings for objects...

Warning message:
“Number of dimensions changing from 50 to 15”



Projecting corrected query onto Reference PCA space

Projecting corrected query onto Reference UMAP space
  |======================================================================| 100%



Creating slots functional.cluster and functional.cluster.conf in query object



In [26]:
query_cd4@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,nCount_originalexp,nFeature_originalexp,barcode,BRI,Run,Well,Batch,⋯,A_Num_Productive_In_Barcode,A_Has_Unproductive,B_Possible_TCR_Doublet,A_Possible_TCR_Doublet,DEMUX_BEST_Category,Hashtag1,Hashtag2,Hashtag3,functional.cluster,functional.cluster.conf
,<fct>,<dbl>,<int>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<chr>,<dbl>
3_AAACCTGAGACCGGAT,3,1443,966,3241,1085,AAACCTGAGACCGGAT-1,2248_hashing,2,1,3,⋯,NaN,nan,nan,nan,SNG,8,19,4305,CD4.Tfh,0.7880882
3_AAACCTGAGAGGGCTT,3,3508,1277,4559,1387,AAACCTGAGAGGGCTT-1,2248_hashing,2,1,3,⋯,1,True,Productive Singleton,Productive Singleton,SNG,391,61,138,CD4.NaiveLike,1.0000000
3_AAACCTGAGAGTCTGG,3,18799,4403,20879,4527,AAACCTGAGAGTCTGG-1,2248_hashing,2,1,3,⋯,1,False,Productive Singleton,Productive Singleton,SNG,9,55,5301,CD4.Th17,0.9560450
3_AAACCTGAGCTAGCCC,3,4133,1617,5523,1737,AAACCTGAGCTAGCCC-1,2248_hashing,2,1,3,⋯,0,True,Productive Singleton,Unproductive Singleton,SNG,1477,50,153,CD4.NaiveLike,0.8356035
3_AAACCTGAGCTGCCCA,3,10485,3080,13193,3205,AAACCTGAGCTGCCCA-1,2248_hashing,2,1,3,⋯,1,False,Productive Singleton,Productive Singleton,SNG,17,39,8338,CD4.NaiveLike,0.6109448
3_AAACCTGAGGCTCATT,3,17306,4230,23923,4363,AAACCTGAGGCTCATT-1,2248_hashing,2,1,3,⋯,1,True,Productive Singleton,Productive Singleton,SNG,26,127,12370,CD4.Th17,1.0000000
3_AAACCTGAGTAGGTGC,3,5169,1668,6440,1776,AAACCTGAGTAGGTGC-1,2248_hashing,2,1,3,⋯,1,False,Productive Singleton,Productive Singleton,SNG,9,1537,99,CD4.NaiveLike,1.0000000
3_AAACCTGAGTATGACA,3,1309,987,2846,1103,AAACCTGAGTATGACA-1,2248_hashing,2,1,3,⋯,NaN,nan,nan,nan,SNG,10,27,3314,CD4.NaiveLike,0.8967498
3_AAACCTGAGTGTACCT,3,9313,2698,11434,2823,AAACCTGAGTGTACCT-1,2248_hashing,2,1,3,⋯,1,True,Productive Singleton,Productive Singleton,SNG,6,47,4892,CD4.Th17,0.6415033


In [27]:
query_cd4@meta.data[c('functional.cluster', 'functional.cluster.conf')] %>% write.table('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.ProjectTILsCD4.tsv',
                              row.names = TRUE, sep = '\t')

In [28]:
query_cd8@meta.data[c('functional.cluster', 'functional.cluster.conf')] %>% write.table('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.ProjectTILsCD8.tsv',
                              row.names = TRUE, sep = '\t')

In [30]:
query_allt@meta.data[c('functional.cluster', 'functional.cluster.conf')] %>% write.table('../../../Data/PerDataset/Stimseq/raw.mincellfilt.htonegfilt.doubletfilt.ProjectTILsAllTMouse.tsv',
                              row.names = TRUE, sep = '\t')